In [29]:
# Get data from GBQ
from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_file(
    '/Users/jaroslawbontruk/Documents/bigqueryexport-183608-b79e75e1593a.json',
)

project_id = "bigqueryexport-183608"

query = """
select supplier_name 
from PayabilitySheets.v_supplier_summary
where payability_status = 'Active'
"""

df = pandas_gbq.read_gbq(query, 
                         project_id = project_id, 
                         credentials = credentials)

In [167]:
# Get data from CSV
import pandas as pd
import numpy as np

df1 = pd.read_csv('500k_vs_no_marketplace_added.xlsx - Sheet1.csv', low_memory = False)
df1.columns = ['all_suppliers', 'selected_suppliers', 'match']

grouped = df1.groupby('selected_suppliers').size().reset_index()
grouped.columns = ['selected_suppliers', 'count']
grouped = grouped.sort_values('count', ascending=False)
grouped

,selected_suppliers,count
1485,Amazon,216
16891,None,136
26136,amazon,115
29104,none,109
20652,Self,90
29792,self,59
17837,Payability,51
15975,Mr.,31
16390,Na,29
16832,No,26


In [208]:
# Prepare URLs (based on CSV data)
uniques = df1.selected_suppliers.unique()
uniques = pd.DataFrame(data = uniques.flatten())
uniques.columns = ['company_name_cleaned']
a = uniques['company_name_cleaned']

uniques = uniques.apply(lambda x: x.str.replace(',',''))
uniques = uniques.apply(lambda x: x.str.replace('!',''))
uniques = uniques.apply(lambda x: x.str.replace("'",""))
uniques = uniques.apply(lambda x: x.str.replace('.',''))
uniques = uniques.apply(lambda x: x.str.replace('&',''))
uniques = uniques.apply(lambda x: x.str.replace(' ','-'))

core_url = "https://www.sellerratings.com/amazon/usa/"
uniques['url'] = core_url + uniques['company_name_cleaned'].astype(str)
uniques['company_name'] = a

print(uniques.loc[29223, 'company_name_cleaned'])
print(uniques.shape)

df = uniques
df

Valentina’s-store
(30882, 3)


,company_name_cleaned,url,company_name
0,Indigo-Socks-LLC,https://www.sellerratings.com/amazon/usa/Indig...,Indigo Socks LLC
1,Find-it-at-the-Bay,https://www.sellerratings.com/amazon/usa/Find-...,Find it at the Bay
2,steph1867,https://www.sellerratings.com/amazon/usa/steph...,steph1867
3,farleysale,https://www.sellerratings.com/amazon/usa/farle...,farleysale
4,PetArtus,https://www.sellerratings.com/amazon/usa/PetArtus,PetArtus
5,Sobr-LVN,https://www.sellerratings.com/amazon/usa/Sobr-LVN,Sobr LVN
6,filefingers,https://www.sellerratings.com/amazon/usa/filef...,filefingers
7,Ware-Design-Works,https://www.sellerratings.com/amazon/usa/Ware-...,Ware Design Works
8,JewelryPlus,https://www.sellerratings.com/amazon/usa/Jewel...,JewelryPlus
9,thebookboys,https://www.sellerratings.com/amazon/usa/thebo...,thebookboys


In [115]:
# Prepare URLs (based on GBQ data)
sample_url = "https://www.sellerratings.com/amazon/usa?name=Soul+Sistahs+Network"
core_url = "https://www.sellerratings.com/amazon/usa?name="

df = df.apply(lambda x: x.str.replace(' ','+'))
df['url'] = core_url + df['supplier_name'].astype(str)

print(df.loc[10, 'url'])
print(df.shape)

https://www.sellerratings.com/amazon/usa?name=positron
(30882, 2)


In [205]:
# Get data from Marketplace Pulse
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
import json

results = pd.DataFrame(columns=['supplier_name', 'url', 'response', 'sellerID', 'reviewCount', 'ratingValue'])
start = time.time()

for i in range(29200, 30881):
    results.loc[i, 'supplier_name'] = df.loc[i, 'company_name']  
    results.loc[i, 'url'] = df.loc[i, 'url'].lower() 
    r = requests.get(df.loc[i, 'url'])
    results.loc[i, 'response'] = str(r)
    if str(r) == '<Response [200]>':
        soup = BeautifulSoup(r.content, 'html.parser')
        for div in soup.find_all(class_='contact'):
            sellerID = div.find('a')['href']
        sellerID = sellerID.replace('https://www.amazon.com/gp/help/contact-seller/contact-seller.html?sellerID=', '')        
        results.loc[i, 'sellerID'] = sellerID
        data = json.loads(soup.find('script', type='application/ld+json').text)
        results.loc[i, 'reviewCount'] = data['aggregateRating']['reviewCount']
        results.loc[i, 'ratingValue'] = data['aggregateRating']['ratingValue']
    if i%10 == 0: print(str(i) + ' URLs in ' + str(time.time() - start) + ' sec')
        
results

29200 URLs in 0.7070257663726807 sec
29210 URLs in 5.825498819351196 sec
29220 URLs in 11.225750923156738 sec
29230 URLs in 16.224901914596558 sec
29240 URLs in 21.494565725326538 sec
29250 URLs in 26.784398078918457 sec
29260 URLs in 32.70924997329712 sec
29270 URLs in 38.24388098716736 sec
29280 URLs in 43.40626788139343 sec
29290 URLs in 48.475919008255005 sec
29300 URLs in 53.76783299446106 sec
29310 URLs in 58.89349889755249 sec
29320 URLs in 64.47310900688171 sec
29330 URLs in 70.1881799697876 sec
29340 URLs in 75.66000485420227 sec
29350 URLs in 81.50590491294861 sec
29360 URLs in 86.74573683738708 sec
29370 URLs in 92.28642296791077 sec
29380 URLs in 97.27395987510681 sec
29390 URLs in 102.77527594566345 sec
29400 URLs in 108.23277592658997 sec
29410 URLs in 114.28166198730469 sec
29420 URLs in 119.99443888664246 sec
29430 URLs in 125.61572670936584 sec
29440 URLs in 130.9175989627838 sec
29450 URLs in 136.66834998130798 sec
29460 URLs in 142.23757195472717 sec
29470 URLs in 14

,supplier_name,url,response,sellerID,reviewCount,ratingValue
29200,V-Tx-SvCs,https://www.sellerratings.com/amazon/usa/v-tx-...,<Response [200]>,A3LT39E7AXP1MB,4,4.0
29201,V.Mae,https://www.sellerratings.com/amazon/usa/vmae,<Response [200]>,A3PQ91OU7VWVYE,1,5.0
29202,V&A,https://www.sellerratings.com/amazon/usa/va,<Response [404]>,NaN,NaN,NaN
29203,v&M deals Corp.,https://www.sellerratings.com/amazon/usa/vm-de...,<Response [404]>,NaN,NaN,NaN
29204,V$E Beauty,https://www.sellerratings.com/amazon/usa/v$e-b...,<Response [404]>,NaN,NaN,NaN
29205,V4fashions,https://www.sellerratings.com/amazon/usa/v4fas...,<Response [404]>,NaN,NaN,NaN
29206,V5NRY Clothing,https://www.sellerratings.com/amazon/usa/v5nry...,<Response [404]>,NaN,NaN,NaN
29207,V9v9Productions,https://www.sellerratings.com/amazon/usa/v9v9p...,<Response [404]>,NaN,NaN,NaN
29208,Va Para Ti,https://www.sellerratings.com/amazon/usa/va-pa...,<Response [404]>,NaN,NaN,NaN
29209,Vaa Sasa LLC,https://www.sellerratings.com/amazon/usa/vaa-s...,<Response [404]>,NaN,NaN,NaN


In [206]:
# Save to CSV
results.to_csv('results.csv', index = False)